## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/jennifermadson/Desktop/Data-Analytics-BootCamp/School-Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cape Town,ZA,-33.9258,18.4232,55.11,73,1,6.60,clear sky
1,1,Qaqortoq,GL,60.7167,-46.0333,32.50,64,21,5.39,few clouds
2,2,Iqaluit,CA,63.7506,-68.5145,31.69,88,100,17.00,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,54.90,71,23,4.90,few clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,56.64,72,27,13.65,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you'd like for your trip? "))
max_temp = float(input("What is the maximum temperature you'd like for your trip? "))


What is the minimum temperature you'd like for your trip? 70
What is the maximum temperature you'd like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
6,6,Saint-Philippe,RE,-21.3585,55.7679,74.75,82,40,13.56,scattered clouds
7,7,Santa Isabel Do Rio Negro,BR,-0.4139,-65.0192,78.01,85,75,1.03,broken clouds
11,11,Ghatanji,IN,20.1333,78.3167,74.08,61,15,3.44,few clouds
14,14,Hithadhoo,MV,-0.6000,73.0833,82.80,73,100,12.75,overcast clouds
15,15,Oum Hadjer,TD,13.2954,19.6966,81.21,25,11,6.73,few clouds
...,...,...,...,...,...,...,...,...,...,...
677,677,Guajara,BR,-2.9667,-57.6667,77.77,85,82,2.77,broken clouds
678,678,Bhuban,IN,20.8833,85.8333,81.90,63,45,2.64,scattered clouds
679,679,Omboue,GA,-1.5746,9.2618,74.98,91,100,3.11,overcast clouds
681,681,Nouakchott,MR,18.0858,-15.9785,75.13,47,24,12.86,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Saint-Philippe,RE,74.75,scattered clouds,-21.3585,55.7679,
7,Santa Isabel Do Rio Negro,BR,78.01,broken clouds,-0.4139,-65.0192,
11,Ghatanji,IN,74.08,few clouds,20.1333,78.3167,
14,Hithadhoo,MV,82.80,overcast clouds,-0.6000,73.0833,
15,Oum Hadjer,TD,81.21,few clouds,13.2954,19.6966,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"  
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    # 6e. Make request and retrieve the JSON data from the search.  
    hotels = requests.get(base_url, params=params).json()
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
#7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df



,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Saint-Philippe,RE,74.75,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Santa Isabel Do Rio Negro,BR,78.01,broken clouds,-0.4139,-65.0192,Hotel GSA
11,Ghatanji,IN,74.08,few clouds,20.1333,78.3167,गजानन माहराज मदील घाटजी
14,Hithadhoo,MV,82.80,overcast clouds,-0.6000,73.0833,Scoop Guest House
21,Kumo,NG,74.52,overcast clouds,10.0447,11.2122,Ajegunle Lodge(corpers Lodge)
...,...,...,...,...,...,...,...
674,Mazamari,PE,78.40,overcast clouds,-11.3258,-74.5308,Hospedaje Ratto
678,Bhuban,IN,81.90,scattered clouds,20.8833,85.8333,Prasanta Sahoo
679,Omboue,GA,74.98,overcast clouds,-1.5746,9.2618,Hotel Olako
681,Nouakchott,MR,75.13,few clouds,18.0858,-15.9785,HOTEL IMAN فندق إيمان


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(10.0, 21.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))